In [1]:
from src.parser.match_recognize_extractor import parse_full_query

if __name__ == "__main__":
    sample_query = """
SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        ONE ROW PER MATCH
        AFTER MATCH SKIP PAST LAST ROW
        PATTERN (AB+ C+ D+)

        DEFINE
            AB AS TRUE,
            C AS totalprice > PREV(totalprice),
            D AS totalprice > PREV(totalprice)
    );

        """


    components = parse_full_query(sample_query)
    print(components)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate ONE ROW PER MATCH AFTER MATCH SKIP PAST LAST ROW PATTERN (AB+ C+ D+) DEFINE AB AS TRUE, C AS totalprice > PREV(totalprice), D AS totalprice > PREV(totalprice) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted ROWS PER MATCH: ONEROWPERMATCH
DEBUG:src.parser.match_recognize_extractor:Extrac

FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=None,
  rows_per_match=ONE ROW PER MATCH,
  after_match_skip=AFTER MATCH SKIP PAST LAST ROW,
  pattern=PatternClause(pattern='AB+ C+ D+', metadata={'variables': ['AB+', 'C+', 'D+'], 'base_variables': ['AB', 'C', 'D']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='AB', condition='TRUE'), Define(variable='C', condition='totalprice>PREV(totalprice)'), Define(variable='D', condition='totalprice>PREV(totalprice)')])
),
  metadata={}
)


In [1]:
from src.parser.match_recognize_extractor import parse_full_query

if __name__ == "__main__":
    sample_query = """
SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES
            A.totalprice AS starting_price,
            LAST(B.totalprice) AS bottom_price,
            LAST(U.totalprice) AS top_price
        ONE ROW PER MATCH
        AFTER MATCH SKIP PAST LAST ROW
        PATTERN (A B+ C+ D+)
        SUBSET U = (C, D)
        DEFINE
            A AS TRUE,
            B AS totalprice < PREV(totalprice),
            C AS totalprice > PREV(totalprice) AND totalprice <= A.totalprice,
            D AS totalprice > PREV(totalprice)
    );

        """


    components = parse_full_query(sample_query)
    print(components)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES A.totalprice AS starting_price, LAST(B.totalprice) AS bottom_price, LAST(U.totalprice) AS top_price ONE ROW PER MATCH AFTER MATCH SKIP PAST LAST ROW PATTERN (A B+ C+ D+) SUBSET U = (C, D) DEFINE A AS TRUE, B AS totalprice < PREV(totalprice), C AS totalprice > PREV(totalprice) AND totalprice <= A.totalprice, D AS totalprice > PREV(totalprice) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortIte

FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False), Measure(expression='LAST(B.totalprice)', alias='bottom_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False), Measure(expression='LAST(U.totalprice)', alias='top_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=ONE ROW PER MATCH,
  after_match_skip=AFTER MATCH SKIP PAST LAST ROW,
  pattern=PatternClause(pattern='A B+ C+ D+', metadata={'variables': ['A', 'B+', 'C+', 'D+'], 'base_variables': ['A', '

In [5]:
from src.parser.match_recognize_extractor import parse_full_query

# List of test queries covering various pattern matching features

test_queries = [
    # 1.1 Basic Concatenation Pattern
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        PATTERN (A B C)
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > PREV(totalprice),
            C AS totalprice < PREV(totalprice)
    );""",

    # 1.2 Alternation Pattern
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES 
            A.totalprice AS a_price,
            B.totalprice AS b_price,
            C.totalprice AS c_price
        PATTERN (A (B | C) D)
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > PREV(totalprice),
            C AS totalprice < PREV(totalprice),
            D AS totalprice > 200
    );""",

    # 1.3 Permutation Pattern
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES 
            A.totalprice AS a_price,
            B.totalprice AS b_price,
            C.totalprice AS c_price
        PATTERN (PERMUTE(A, B, C))
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > 200,
            C AS totalprice > 300
    );""",

    # 1.4 Complex Grouping and Quantifiers
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        PATTERN ((A B){2,3} C* D+? E?)
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > PREV(totalprice),
            C AS totalprice < PREV(totalprice),
            D AS totalprice > A.totalprice,
            E AS totalprice > 500
    );""",

    # 1.5 Pattern with Anchors
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        PATTERN (^A B C$)
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > PREV(totalprice),
            C AS totalprice < PREV(totalprice)
    );""",

    # 1.6 Pattern with Exclusion Syntax
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS price
        ALL ROWS PER MATCH
        PATTERN (A {-B C-} D)
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > PREV(totalprice),
            C AS totalprice < PREV(totalprice),
            D AS totalprice > 200
    );""",

    # 2.1 ONE ROW PER MATCH (Default)
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        ONE ROW PER MATCH
        PATTERN (A B+ C?)
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > PREV(totalprice)
    );""",

    # 2.2 ALL ROWS PER MATCH Variants (Omit Empty Matches)
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        ALL ROWS PER MATCH OMIT EMPTY MATCHES
        PATTERN (A B+ C?)
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > PREV(totalprice)
    );""",

    # 2.2 ALL ROWS PER MATCH Variants (With Unmatched Rows)
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        ALL ROWS PER MATCH WITH UNMATCHED ROWS
        PATTERN (A B+ C?)
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > PREV(totalprice)
    );""",

    # 3.1 AFTER MATCH SKIP TO NEXT ROW
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        AFTER MATCH SKIP TO NEXT ROW
        PATTERN (A B+ C?)
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > PREV(totalprice)
    );""",

    # 3.1 AFTER MATCH SKIP TO LAST B
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        AFTER MATCH SKIP TO LAST B
        PATTERN (A B+ C?)
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > PREV(totalprice)
    );""",

    # 4.1 Single SUBSET Definition
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES 
            A.totalprice AS a_price,
            U.totalprice AS u_price
        PATTERN (A B+ C+)
        SUBSET U = (B, C)
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > PREV(totalprice),
            C AS totalprice < PREV(totalprice)
    );""",

    # 4.1 Multiple SUBSET Definitions
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES 
            A.totalprice AS a_price,
            U.totalprice AS u_price,
            V.totalprice AS v_price
        PATTERN (A B+ C+ D+)
        SUBSET 
            U = (A, B),
            V = (C, D)
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > PREV(totalprice),
            C AS totalprice < PREV(totalprice),
            D AS totalprice > 200
    );""",

    # 5.2 Nested Navigation Functions
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES 
            PREV(FIRST(B.totalprice), 1) AS prev_first_b
        PATTERN (A B+ C?)
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > PREV(totalprice)
    );""",

    # 5.3 RUNNING and FINAL Semantics
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES 
            RUNNING COUNT(*) AS running_count,
            FINAL COUNT(*) AS final_count,
            RUNNING LAST(B.totalprice) AS running_last_b,
            FINAL LAST(B.totalprice) AS final_last_b
        PATTERN (A B+ C?)
        DEFINE 
            A AS totalprice > 100,
            B AS totalprice > PREV(totalprice)
    );""",



    # 7. Complex Real-World Pattern (V-Shape Example)
    """SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES
            A.totalprice AS starting_price,
            LAST(B.totalprice) AS bottom_price,
            LAST(U.totalprice) AS top_price
        ONE ROW PER MATCH
        AFTER MATCH SKIP PAST LAST ROW
        PATTERN (A B+ C+ D+)
        SUBSET U = (C, D)
        DEFINE
            A AS TRUE,
            B AS totalprice < PREV(totalprice),
            C AS totalprice > PREV(totalprice) AND totalprice <= A.totalprice,
            D AS totalprice > PREV(totalprice)
    );"""
]

if __name__ == "__main__":
    for i, query in enumerate(test_queries, start=1):
        print(f"--- Test Case {i} ---")
        components = parse_full_query(query)
        print("Parsed Components:")
        print(components)
        print()


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES A.totalprice AS starting_price PATTERN (A B C) DEFINE A AS totalprice > 100, B AS totalprice > PREV(totalprice), C AS totalprice < PREV(totalprice) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='A.totalprice', alias='start

--- Test Case 1 ---
Parsed Components:
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=None,
  after_match_skip=None,
  pattern=PatternClause(pattern='A B C', metadata={'variables': ['A', 'B', 'C'], 'base_variables': ['A', 'B', 'C']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='A', condition='totalprice>100'), Define(variable='B', condition='totalprice>PREV(totalprice)'), Define(variable='C', condition='totalprice<PREV(totalprice)')])
),
  metadata={}
)

--- Test Case 2 ---
Parsed Components:


DEBUG:src.parser.match_recognize_extractor:Extracted variables from define condition 'totalprice>PREV(totalprice)': []
DEBUG:src.parser.match_recognize_extractor:Pattern variables: {'D', 'A', 'B', 'C'}
DEBUG:src.parser.match_recognize_extractor:Referenced variables: {'U', 'A', 'C', 'D', 'B'}
DEBUG:src.parser.match_recognize_extractor:Required variables: {'D', 'A', 'B', 'C'}
DEBUG:src.parser.match_recognize_extractor:Defined variables: {'D', 'A', 'B', 'C'}
DEBUG:src.parser.match_recognize_extractor:Subset variables: {'U': ['C', 'D']}
DEBUG:src.parser.match_recognize_extractor:Validated function usage for measure: A.totalprice
DEBUG:src.parser.match_recognize_extractor:Validated function usage for measure: LAST(B.totalprice)
DEBUG:src.parser.match_recognize_extractor:Validated function usage for measure: LAST(U.totalprice)
DEBUG:src.parser.match_recognize_extractor:Extracted MATCH_RECOGNIZE clause via recursive search.


Parsed Components:
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False), Measure(expression='LAST(B.totalprice)', alias='bottom_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False), Measure(expression='LAST(U.totalprice)', alias='top_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=ONE ROW PER MATCH,
  after_match_skip=AFTER MATCH SKIP PAST LAST ROW,
  pattern=PatternClause(pattern='A B+ C+ D+', metadata={'variables': ['A', 'B+', 'C+', 'D+'], 'base_

In [3]:
from src.parser.match_recognize_extractor import parse_full_query
from src.ast.ast_nodes import FullQueryAST

# Define a list of test queries to cover edge cases and full coverage
test_queries = [
    # 1. Basic ONE ROW PER MATCH
    {
        "name": "Basic One Row Per Match",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price
                ONE ROW PER MATCH
                PATTERN (A B+)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 2. ALL ROWS PER MATCH with SHOW EMPTY MATCHES, SUBSET clause and union variable
    {
        "name": "All Rows Per Match - Show Empty Matches",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price,
                    LAST(U.totalprice) AS top_price
                ALL ROWS PER MATCH SHOW EMPTY MATCHES
                PATTERN (A | B | C | D)
                SUBSET U = (C, D)
                DEFINE
                    B AS totalprice < PREV(totalprice),
                    C AS totalprice > PREV(totalprice) AND totalprice <= A.totalprice,
                    D AS totalprice > PREV(totalprice)
            );
        """
    },
    # 3. ALL ROWS PER MATCH with OMIT EMPTY MATCHES
    {
        "name": "All Rows Per Match - Omit Empty Matches",
        "query": """
            SELECT * FROM transactions MATCH_RECOGNIZE(
                PARTITION BY account_id
                ORDER BY transaction_date
                MEASURES
                    A.amount AS start_amount,
                    C.amount AS final_amount
                ALL ROWS PER MATCH OMIT EMPTY MATCHES
                PATTERN (A | B |C)
                DEFINE
                    B AS amount < PREV(amount),
                    C AS amount > PREV(amount)
            );
        """
    },
    # 4. ALL ROWS PER MATCH WITH UNMATCHED ROWS
    {
        "name": "All Rows Per Match - With Unmatched Rows",
        "query": """
            SELECT * FROM sales MATCH_RECOGNIZE(
                PARTITION BY region
                ORDER BY sale_date
                MEASURES
                    A.price AS initial_price,
                    D.price AS final_price,
                    MATCH_NUMBER() AS match_id
                ALL ROWS PER MATCH WITH UNMATCHED ROWS
                PATTERN (A B C D)
                DEFINE
                    B AS price < PREV(price),
                    C AS price > PREV(price),
                    D AS price > A.price
            );
        """
    },
    # 5. Nested Pattern: union variables and repeated groups with WITH UNMATCHED ROWS
    {
        "name": "Nested Pattern with Unmatched Rows",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    F.totalprice AS ending_price
                ALL ROWS PER MATCH WITH UNMATCHED ROWS
                PATTERN (A (B C | D E)+ F)
                DEFINE
                    B AS totalprice < PREV(totalprice),
                    C AS totalprice > PREV(totalprice),
                    D AS totalprice = PREV(totalprice),
                    E AS totalprice >= PREV(totalprice)
            );
        """
    },
    # 6. Using Classifier() and MATCH_NUMBER() functions
    {
        "name": "Classifier and Match Number Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    CLASSIFIER() AS pattern_type,
                    MATCH_NUMBER() AS match_id,
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 7. Running vs Final Semantics in MEASURES
    {
        "name": "Running vs Final Semantics in Measures",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    RUNNING LAST(A.totalprice) AS running_last_price,
                    FINAL LAST(A.totalprice) AS final_last_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 8. Aggregates with Running vs Final semantics (avg and count)
    {
        "name": "Running Avg vs Final Count Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    RUNNING avg(A.totalprice) AS running_avg,
                    FINAL count(A.*) AS final_count
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 9. Empty Match: pattern that produces an empty match for every row
    {
        "name": "Empty Match and Unmatched Rows",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    MATCH_NUMBER() AS match_id
                ONE ROW PER MATCH
                PATTERN (())
            );
        """
    },
    # 10. Physical Navigation Functions: PREV() and NEXT()
    {
        "name": "Physical Navigation Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    PREV(A.totalprice, 2) AS prev_price,
                    NEXT(A.totalprice, 1) AS next_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 11. Nested Navigation Function
    {
        "name": "Nested Navigation Function",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    PREV(FIRST(A.totalprice, 3), 2) AS nested_nav_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # --- Additional Edge Cases ---
    # 12. Unconventional Whitespace and Nested Functions in Complex Pattern
    {
        "name": "Complex Pattern with Unconventional Whitespace",
        "query": """
            SELECT *
            FROM   orders
            MATCH_RECOGNIZE(
                PARTITION BY   custkey
                ORDER BY   orderdate
                MEASURES
                  PREV(  FIRST( A.totalprice , 3 )  , 2 ) AS nested_nav_price
                ONE ROW PER MATCH
                PATTERN (  A   B+   C )
                DEFINE
                  B AS    totalprice < PREV(  totalprice )
            );
        """
    },
    # 13. Missing ORDER BY clause (should raise an error)
    {
        "name": "Error: Missing ORDER BY Clause",
        "query": """
            SELECT *
            FROM orders
            MATCH_RECOGNIZE(
                PARTITION BY custkey
                MEASURES
                    A.totalprice AS starting_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 14. Missing PATTERN clause when DEFINE is provided (should raise an error)
    {
        "name": "Error: Missing PATTERN Clause",
        "query": """
            SELECT *
            FROM orders
            MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 15. Malformed PATTERN clause (missing closing parenthesis)
    {
        "name": "Error: Malformed PATTERN Clause",
        "query": """
            SELECT *
            FROM orders
            MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price
                ONE ROW PER MATCH
                PATTERN (A B+ C
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 16. Malformed DEFINE clause (missing 'AS' keyword)
    {
        "name": "Error: Malformed DEFINE Clause",
        "query": """
            SELECT *
            FROM orders
            MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B totalprice < PREV(totalprice)
            );
        """
    }
]

# Run all test cases and print the resulting AST or error message
for test in test_queries:
    print(f"🔹 Running Test: {test['name']}")
    try:
        query_ast = parse_full_query(test["query"])
        print(query_ast)
    except Exception as e:
        print(f"Error: {e}")
    print("=" * 80)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES A.totalprice AS starting_price, LAST(B.totalprice) AS bottom_price ONE ROW PER MATCH PATTERN (A B+) DEFINE B AS totalprice < PREV(totalprice) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_pr

🔹 Running Test: Basic One Row Per Match
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False), Measure(expression='LAST(B.totalprice)', alias='bottom_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=ONE ROW PER MATCH,
  after_match_skip=None,
  pattern=PatternClause(pattern='A B+', metadata={'variables': ['A', 'B+'], 'base_variables': ['A', 'B']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='B', condition='totalprice<PREV(totalprice)')])
),
  metadata={}
)
🔹 Runn

In [4]:
from src.parser.match_recognize_extractor import parse_full_query

# Define a list of test cases with descriptions and query strings.
sample_queries = [
    {
        "description": "Simple pattern with whitespace",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (A b+ c*)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
);
"""
    },
    {
        "description": "Pattern without whitespace",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (Ab+c*)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
);
"""
    },
    {
        "description": "Single word pattern (one token)",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES vfggf.totalprice AS starting_price
    PATTERN (vfggf)
    DEFINE vfggf AS totalprice > 0
);
"""
    },
    {
        "description": "Pattern with PERMUTE construct",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES LAST(U.totalprice) AS top_price
    PATTERN (PERMUTE(A,B,C))
    SUBSET U = (A,B,C)
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    },
    {
        "description": "Pattern with grouping and quantifiers",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN ((A B+ C+))
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    },
    {
        "description": "Mixed pattern: whitespace vs. no whitespace",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (A b+ c*)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
);
"""
    },
    {
        "description": "Pattern with quantifiers (greedy)",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
    PATTERN (A{2,4})
    DEFINE A AS totalprice > 0
);
"""
    },
    {
        "description": "Pattern with quantifiers (reluctant)",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
    PATTERN (A{2,4}?)
    DEFINE A AS totalprice > 0
);
"""
    },
    {
        "description": "Empty PATTERN clause (allowed as empty match)",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES NULL AS starting_price
    PATTERN ()
    DEFINE A AS totalprice > 0
);
"""
    },
    {
        "description": "Pattern with irregular whitespace and tabs",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY   custkey
    ORDER BY    orderdate
    MEASURES   A.totalprice   AS   starting_price
    PATTERN (   A   \t   b+   \t  c*   )
    DEFINE   A   AS totalprice > 0,   b   AS totalprice < PREV(totalprice),   c   AS totalprice < 100
);
"""
    },
    {
        "description": "Error: Undefined variable in DEFINE (missing c)",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (A b+ c*)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice)
);
"""
    },
    {
        "description": "Error: Extra variable in DEFINE (c extra)",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    PATTERN (A b+)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
);
"""
    },
    {
        "description": "Pattern with bounded quantifiers preserving commas",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (A{2,4} B+)
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice)
);
"""
    },
    {
        "description": "Complex nested grouping with mixed quantifiers",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (((AB+)(CD{2,4}?))E+)
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice > 50, D AS totalprice < 100, E AS totalprice BETWEEN 50 AND 150
);
"""
    },
    {
        "description": "Error: Undefined variable in DEFINE (extra variable missing in pattern)",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    PATTERN (A b+ c*)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice)
);
"""
    }
]

# Run test cases
for test in sample_queries:
    print(f"--- Running Test: {test['description']} ---")
    try:
        result = parse_full_query(test["query"])
        print("Test Passed: Query parsed successfully.")
        print(result)
    except Exception as e:
        print("Test Failed with error:")
        print(e)
    print("=" * 80)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES A.totalprice AS starting_price PATTERN (A b+ c*) DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', m

--- Running Test: Simple pattern with whitespace ---
Test Passed: Query parsed successfully.
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=None,
  after_match_skip=None,
  pattern=PatternClause(pattern='A b+ c*', metadata={'variables': ['A', 'b+', 'c*'], 'base_variables': ['A', 'b', 'c']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='A', condition='totalprice>0'), Define(variable='b', condition='totalprice<PREV(totalprice)'), Define(variable='c', condition='totalprice<100')])
),
  metadata={}

In [5]:
from src.parser.match_recognize_extractor import parse_full_query

def run_test(query, description):
    print(f"--- Running Test: {description} ---")
    try:
        ast = parse_full_query(query)
        print("Test Passed: Query parsed successfully.")
        print(ast)
    except Exception as e:
        print("Test Failed with error:")
        print(e)
    print("="*80)

if __name__ == "__main__":
    # 1. Alternation using '|' operator
    test_query1 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (A | B | C)
    DEFINE A AS totalprice > 100, B AS totalprice < 50, C AS totalprice BETWEEN 50 AND 100
);
"""
    run_test(test_query1, "Alternate forms: Alternation using '|' operator")

    # 2. Grouping
    test_query2 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN ((A B) C)
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    run_test(test_query2, "Alternate forms: Grouping")

    # 3. Permutation
    test_query3 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES LAST(U.totalprice) AS top_price
    PATTERN (PERMUTE(A, B, C))
    SUBSET U = (A, B, C)
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    run_test(test_query3, "Alternate forms: Permutation")

    # 4. Quantifiers with bounds (Greedy)
    test_query4a = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
    PATTERN (A{2,4})
    DEFINE A AS totalprice > 0
);
"""
    run_test(test_query4a, "Quantifiers with bounds: Greedy")

    # 4. Quantifiers with bounds (Reluctant)
    test_query4b = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
    PATTERN (A{2,4}?)
    DEFINE A AS totalprice > 0
);
"""
    run_test(test_query4b, "Quantifiers with bounds: Reluctant")

    # 5. Valid Navigation and Aggregates
    test_query5 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES LAST(A.totalprice) AS final_price, avg(A.totalprice) AS avg_price
    PATTERN (A+)
    DEFINE A AS totalprice > 0
);
"""
    run_test(test_query5, "Valid Navigation and Aggregates")

    # 6. Invalid Navigation Function (should fail)
    test_query6 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES LAST(1) AS invalid_nav
    PATTERN (A+)
    DEFINE A AS totalprice > 0
);
"""
    run_test(test_query6, "Invalid Navigation Function (should fail)")

    # 7. Invalid Nested Aggregate in Navigation (should fail)
    test_query7 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES FIRST(avg(A.totalprice),2) AS invalid_agg_nav
    PATTERN (A+)
    DEFINE A AS totalprice > 0
);
"""
    run_test(test_query7, "Invalid Nested Aggregate in Navigation (should fail)")

    # 8. Case-sensitive identifier validation
    test_query8 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (A b+ c)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
);
"""
    run_test(test_query8, "Case-sensitive identifier validation")

    # 9. Grouping with Quantifiers
    test_query9 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN ((A B+ C+))
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    run_test(test_query9, "Alternate forms: Grouping with quantifiers")

    # 10. Aggregate with inconsistent variable references (should fail)
    test_query10 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES max_by(A.totalprice, B.totalprice) AS invalid_agg
    PATTERN (A B+ C)
    DEFINE B AS totalprice < PREV(totalprice)
);
"""
    run_test(test_query10, "Aggregate with inconsistent variable references (should fail)")

    # 11. Aggregate with consistent variable references
    test_query11 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES avg(A.totalprice) AS avg_price, count(A.*) AS count_price
    PATTERN (A B+ C)
    DEFINE B AS totalprice < PREV(totalprice)
);
"""
    run_test(test_query11, "Aggregate with consistent variable references")

    # 12. Navigation and Aggregate with MATCH_NUMBER and avg
    test_query12 = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES MATCH_NUMBER() AS match_id, avg(A.totalprice) AS avg_price
    PATTERN (A B+ C)
    DEFINE B AS totalprice < PREV(totalprice)
);
"""
    run_test(test_query12, "MATCH_NUMBER with aggregates")


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES A.totalprice AS starting_price PATTERN (A | B | C) DEFINE A AS totalprice > 100, B AS totalprice < 50, C AS totalprice BETWEEN 50 AND 100 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price'

--- Running Test: Alternate forms: Alternation using '|' operator ---
Test Passed: Query parsed successfully.
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=None,
  after_match_skip=None,
  pattern=PatternClause(pattern='A | B | C', metadata={'variables': ['A', 'B', 'C'], 'base_variables': ['A', 'B', 'C']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='A', condition='totalprice>100'), Define(variable='B', condition='totalprice<50'), Define(variable='C', condition='totalpriceBETWEEN50AND100')])


In [6]:
from src.parser.match_recognize_extractor import parse_full_query

def run_test(query: str, description: str):
    print(f"--- Running Test: {description} ---")
    try:
        ast = parse_full_query(query)
        print("Test Passed: Query parsed successfully.")
        print(ast)
    except Exception as e:
        print("Test Failed with error:")
        print(e)
    print("=" * 80)

if __name__ == "__main__":
    # 1. Alternate forms: Alternation using '|' operator
    query1 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        PATTERN (A | B | C)
        DEFINE A AS totalprice > 100, B AS totalprice < 50, C AS totalprice BETWEEN 50 AND 100
    );
    """
    run_test(query1, "Alternate forms: Alternation using '|' operator")

    # 2. Alternate forms: Grouping (using parentheses)
    query2 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        PATTERN ((A B C))
        DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
    );
    """
    run_test(query2, "Alternate forms: Grouping")

    # 3. Alternate forms: Permutation
    query3 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES LAST(U.totalprice) AS top_price
        PATTERN (PERMUTE(A, B, C))
        SUBSET U = (A, B, C)
        DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
    );
    """
    run_test(query3, "Alternate forms: Permutation")

    # 4a. Quantifiers with bounds: Greedy
    query4a = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
        PATTERN (A{2,4})
        DEFINE A AS totalprice > 0
    );
    """
    run_test(query4a, "Quantifiers with bounds: Greedy")

    # 4b. Quantifiers with bounds: Reluctant
    query4b = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
        PATTERN (A{2,4}?)
        DEFINE A AS totalprice > 0
    );
    """
    run_test(query4b, "Quantifiers with bounds: Reluctant")

    # 5. Valid Navigation and Aggregates
    query5 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES LAST(A.totalprice) AS final_price, avg(A.totalprice) AS avg_price
        PATTERN (A+)
        DEFINE A AS totalprice > 0
    );
    """
    run_test(query5, "Valid Navigation and Aggregates")

    # 6. Invalid Navigation Function (should fail)
    query6 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES LAST(1) AS invalid_nav
        PATTERN (A+)
        DEFINE A AS totalprice > 0
    );
    """
    run_test(query6, "Invalid Navigation Function (should fail)")

    # 7. Invalid Nested Aggregate in Navigation (should fail)
    query7 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES FIRST(avg(A.totalprice),2) AS invalid_agg_nav
        PATTERN (A+)
        DEFINE A AS totalprice > 0
    );
    """
    run_test(query7, "Invalid Nested Aggregate in Navigation (should fail)")

    # 8. Case-Sensitive Identifier Validation
    query8 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        PATTERN (A b+ c)
        DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
    );
    """
    run_test(query8, "Case-Sensitive Identifier Validation")

    # 9. Grouping with Quantifiers
    query9 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES A.totalprice AS starting_price
        PATTERN ((A B+ C+))
        DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
    );
    """
    run_test(query9, "Grouping with Quantifiers")

    # 10. Aggregate with inconsistent variable references (should fail)
    query10 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES max_by(A.totalprice, B.totalprice) AS invalid_agg
        PATTERN (A B+ C)
        DEFINE B AS totalprice < PREV(totalprice)
    );
    """
    run_test(query10, "Aggregate with Inconsistent Variable References (should fail)")

    # 11. Aggregate with consistent variable references
    query11 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES avg(A.totalprice) AS avg_price, count(A.*) AS count_price
        PATTERN (A B+ C)
        DEFINE B AS totalprice < PREV(totalprice), A AS TRUE, C AS TRUE
    );
    """
    run_test(query11, "Aggregate with Consistent Variable References")

    # 12. MATCH_NUMBER with aggregates
    query12 = """
    SELECT * FROM orders MATCH_RECOGNIZE(
        PARTITION BY custkey
        ORDER BY orderdate
        MEASURES MATCH_NUMBER() AS match_id, avg(A.totalprice) AS avg_price
        PATTERN (A B+ C)
        DEFINE B AS totalprice < PREV(totalprice), A AS TRUE, C AS TRUE
    );
    """
    run_test(query12, "MATCH_NUMBER with Aggregates")


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES A.totalprice AS starting_price PATTERN (A | B | C) DEFINE A AS totalprice > 100, B AS totalprice < 50, C AS totalprice BETWEEN 50 AND 100 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price'

--- Running Test: Alternate forms: Alternation using '|' operator ---
Test Passed: Query parsed successfully.
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=None,
  after_match_skip=None,
  pattern=PatternClause(pattern='A | B | C', metadata={'variables': ['A', 'B', 'C'], 'base_variables': ['A', 'B', 'C']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='A', condition='totalprice>100'), Define(variable='B', condition='totalprice<50'), Define(variable='C', condition='totalpriceBETWEEN50AND100')])


In [7]:
from src.parser.match_recognize_extractor import parse_match_recognize_query

# Define a list of test cases with a description and query string.
sample_queries = [
    {
        "description": "Simple pattern with whitespace",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (A b+ c*)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
);
"""
    },
    {
        "description": "Pattern without whitespace",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (Ab+c*)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
);
"""
    },
    {
        "description": "Single word pattern (one token)",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES vfggf.totalprice AS starting_price
    PATTERN (vfggf)
    DEFINE vfggf AS totalprice > 0
);
"""
    },
    {
        "description": "Pattern with PERMUTE construct",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES LAST(U.totalprice) AS top_price
    PATTERN (PERMUTE(A,B,C))
    SUBSET U = (A,B,C)
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    },
    {
        "description": "Pattern with grouping and quantifiers",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN ((A B+ C+))
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    },
    {
        "description": "Mixed pattern: whitespace vs. no whitespace",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (A b+ c*)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
);
"""
    },
    {
        "description": "Pattern with quantifiers (greedy)",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
    PATTERN (A{2,4})
    DEFINE A AS totalprice > 0
);
"""
    },
    {
        "description": "Pattern with quantifiers (reluctant)",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
    PATTERN (A{2,4}?)
    DEFINE A AS totalprice > 0
);
"""
    }
]

# Iterate over each test case.
for test in sample_queries:
    print(f"--- Running Test: {test['description']} ---")
    try:
        result = parse_match_recognize_query(test["query"])
        print("Test Passed: Query parsed successfully.")
        print(result)
    except Exception as e:
        print("Test Failed with error:")
        print(e)
    print("=" * 80)


DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)])
DEBUG:src.parser.match_recognize_extractor:Updated Pattern tokens: {'variables': ['A', 'b+', 'c*'], 'base_variables': ['A', 'b', 'c']}
DEBUG:src.parser.match_recognize_extractor:PATTERN clause validated successfully: A b+ c*
DEBUG:src.parser.match_recognize_extractor:Extracted Pattern: PatternClause(pattern='A b+ c*', metadata={'variables': ['A', 'b+', 'c*'], 'base_variables': ['A', 'b', 'c']})
DEBUG:src.parser.match_recognize

--- Running Test: Simple pattern with whitespace ---
Test Passed: Query parsed successfully.
MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=None,
  after_match_skip=None,
  pattern=PatternClause(pattern='A b+ c*', metadata={'variables': ['A', 'b+', 'c*'], 'base_variables': ['A', 'b', 'c']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='A', condition='totalprice>0'), Define(variable='b', condition='totalprice<PREV(totalprice)'), Define(variable='c', condition='totalprice<100')])
)
--- Running Test: Pattern without whitespace ---
Test Passed: Query parsed successfully.
MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),

In [8]:
from src.parser.match_recognize_extractor import parse_match_recognize_query

sample_queries = [
    {
        "description": "Simple pattern with whitespace",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (A b+ c*)
    DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
);
"""
    },
    {
        "description": "Pattern without whitespace",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN (AB+C*)
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    },
    {
        "description": "Single word pattern (one token)",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES vfggf.totalprice AS starting_price
    PATTERN (vfggf)
    DEFINE vfggf AS totalprice > 0
);
"""
    },
    {
        "description": "Pattern with PERMUTE construct",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES LAST(U.totalprice) AS top_price
    PATTERN (PERMUTE(A,B,C))
    SUBSET U = (A,B,C)
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    },
    {
        "description": "Pattern with grouping and quantifiers",
        "query": """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY custkey
    ORDER BY orderdate
    MEASURES A.totalprice AS starting_price
    PATTERN ((A B+ C+))
    DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
);
"""
    }
]

for test in sample_queries:
    print(f"--- Running Test: {test['description']} ---")
    try:
        result = parse_match_recognize_query(test["query"])
        print("Test Passed: Query parsed successfully.")
        print(result)
    except Exception as e:
        print("Test Failed with error:")
        print(e)
    print("=" * 80)


DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)])
DEBUG:src.parser.match_recognize_extractor:Updated Pattern tokens: {'variables': ['A', 'b+', 'c*'], 'base_variables': ['A', 'b', 'c']}
DEBUG:src.parser.match_recognize_extractor:PATTERN clause validated successfully: A b+ c*
DEBUG:src.parser.match_recognize_extractor:Extracted Pattern: PatternClause(pattern='A b+ c*', metadata={'variables': ['A', 'b+', 'c*'], 'base_variables': ['A', 'b', 'c']})
DEBUG:src.parser.match_recognize

--- Running Test: Simple pattern with whitespace ---
Test Passed: Query parsed successfully.
MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=None,
  after_match_skip=None,
  pattern=PatternClause(pattern='A b+ c*', metadata={'variables': ['A', 'b+', 'c*'], 'base_variables': ['A', 'b', 'c']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='A', condition='totalprice>0'), Define(variable='b', condition='totalprice<PREV(totalprice)'), Define(variable='c', condition='totalprice<100')])
)
--- Running Test: Pattern without whitespace ---
Test Passed: Query parsed successfully.
MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),

In [9]:
from src.parser.match_recognize_extractor import parse_full_query
from src.ast.ast_nodes import FullQueryAST

# Define a list of test queries to cover edge cases and full coverage
test_queries = [
    # 1. Basic ONE ROW PER MATCH
    {
        "name": "Basic One Row Per Match",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 2. ALL ROWS PER MATCH with SHOW EMPTY MATCHES, SUBSET clause and union variable
    {
        "name": "All Rows Per Match - Show Empty Matches",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price,
                    LAST(U.totalprice) AS top_price
                ALL ROWS PER MATCH SHOW EMPTY MATCHES
                PATTERN (A | B | C | D)
                SUBSET U = (C, D)
                DEFINE
                    B AS totalprice < PREV(totalprice),
                    C AS totalprice > PREV(totalprice) AND totalprice <= A.totalprice,
                    D AS totalprice > PREV(totalprice)
            );
        """
    },
    # 3. ALL ROWS PER MATCH with OMIT EMPTY MATCHES
    {
        "name": "All Rows Per Match - Omit Empty Matches",
        "query": """
            SELECT * FROM transactions MATCH_RECOGNIZE(
                PARTITION BY account_id
                ORDER BY transaction_date
                MEASURES
                    A.amount AS start_amount,
                    C.amount AS final_amount
                ALL ROWS PER MATCH OMIT EMPTY MATCHES
                PATTERN (A | B | C)
                DEFINE
                    B AS amount < PREV(amount),
                    C AS amount > PREV(amount)
            );
        """
    },
    # 4. ALL ROWS PER MATCH WITH UNMATCHED ROWS
    {
        "name": "All Rows Per Match - With Unmatched Rows",
        "query": """
            SELECT * FROM sales MATCH_RECOGNIZE(
                PARTITION BY region
                ORDER BY sale_date
                MEASURES
                    A.price AS initial_price,
                    D.price AS final_price
                ALL ROWS PER MATCH WITH UNMATCHED ROWS
                PATTERN (A B C D)
                DEFINE
                    B AS price < PREV(price),
                    C AS price > PREV(price),
                    D AS price > A.price
            );
        """
    },
    # 5. Nested Pattern: union variables and repeated groups with WITH UNMATCHED ROWS
    {
        "name": "Nested Pattern with Unmatched Rows",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    F.totalprice AS ending_price
                ALL ROWS PER MATCH WITH UNMATCHED ROWS
                PATTERN (A (B C | D E)+ F)
                DEFINE
                    B AS totalprice < PREV(totalprice),
                    C AS totalprice > PREV(totalprice),
                    D AS totalprice = PREV(totalprice),
                    E AS totalprice >= PREV(totalprice)
            );
        """
    },
    # 6. Using Classifier() and MATCH_NUMBER() functions
    {
        "name": "Classifier and Match Number Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    CLASSIFIER() AS pattern_type,
                    MATCH_NUMBER() AS match_id,
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 7. Running vs Final Semantics in MEASURES
    {
        "name": "Running vs Final Semantics in Measures",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    RUNNING LAST(A.totalprice) AS running_last_price,
                    FINAL LAST(A.totalprice) AS final_last_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 8. Aggregates with Running vs Final semantics (avg and count)
    {
        "name": "Running Avg vs Final Count Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    RUNNING avg(A.totalprice) AS running_avg,
                    FINAL count(A.*) AS final_count
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 9. Empty Match: pattern that produces an empty match for every row
    {
        "name": "Empty Match and Unmatched Rows",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    count(A.*) AS match_count
                ONE ROW PER MATCH
                PATTERN (())
            );
        """
    },
    # 10. Physical Navigation Functions: PREV() and NEXT()
    {
        "name": "Physical Navigation Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    PREV(A.totalprice, 2) AS prev_price,
                    NEXT(A.totalprice, 1) AS next_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 11. Nested Navigation Function
    {
        "name": "Nested Navigation Function",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    PREV(FIRST(A.totalprice, 3), 2) AS nested_nav_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    }
]

# Run all test cases and print the resulting AST
for test in test_queries:
    print(f"🔹 Running Test: {test['name']}")
    try:
        query_ast = parse_full_query(test["query"])
        print(query_ast)
    except Exception as e:
        print(f"Error: {e}")
    print("=" * 80)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES A.totalprice AS starting_price, LAST(B.totalprice) AS bottom_price ONE ROW PER MATCH PATTERN (A B+ C) DEFINE B AS totalprice < PREV(totalprice) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_

🔹 Running Test: Basic One Row Per Match
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False), Measure(expression='LAST(B.totalprice)', alias='bottom_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=ONE ROW PER MATCH,
  after_match_skip=None,
  pattern=PatternClause(pattern='A B+ C', metadata={'variables': ['A', 'B+', 'C'], 'base_variables': ['A', 'B', 'C']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='B', condition='totalprice<PREV(totalprice)')])
),
  metadata

In [10]:
from src.parser.match_recognize_extractor import parse_full_query
from src.ast.ast_nodes import FullQueryAST

# Define a list of test queries to cover edge cases and full coverage
test_queries = [
    # 1. Basic ONE ROW PER MATCH
    {
        "name": "Basic One Row Per Match",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 2. ALL ROWS PER MATCH with SHOW EMPTY MATCHES, SUBSET clause and union variable
    {
        "name": "All Rows Per Match - Show Empty Matches",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price,
                    LAST(U.totalprice) AS top_price
                ALL ROWS PER MATCH SHOW EMPTY MATCHES
                PATTERN (A | B | C | D)
                SUBSET U = (C, D)
                DEFINE
                    B AS totalprice < PREV(totalprice),
                    C AS totalprice > PREV(totalprice) AND totalprice <= A.totalprice,
                    D AS totalprice > PREV(totalprice)
            );
        """
    },
    # 3. ALL ROWS PER MATCH with OMIT EMPTY MATCHES
    {
        "name": "All Rows Per Match - Omit Empty Matches",
        "query": """
            SELECT * FROM transactions MATCH_RECOGNIZE(
                PARTITION BY account_id
                ORDER BY transaction_date
                MEASURES
                    A.amount AS start_amount,
                    C.amount AS final_amount
                ALL ROWS PER MATCH OMIT EMPTY MATCHES
                PATTERN (A | B | C)
                DEFINE
                    B AS amount < PREV(amount),
                    C AS amount > PREV(amount)
            );
        """
    },
    # 4. ALL ROWS PER MATCH WITH UNMATCHED ROWS
    {
        "name": "All Rows Per Match - With Unmatched Rows",
        "query": """
            SELECT * FROM sales MATCH_RECOGNIZE(
                PARTITION BY region
                ORDER BY sale_date
                MEASURES
                    A.price AS initial_price,
                    D.price AS final_price
                ALL ROWS PER MATCH WITH UNMATCHED ROWS
                PATTERN (A B C D)
                DEFINE
                    B AS price < PREV(price),
                    C AS price > PREV(price),
                    D AS price > A.price
            );
        """
    },
    # 5. Nested Pattern: union variables and repeated groups with WITH UNMATCHED ROWS
    {
        "name": "Nested Pattern with Unmatched Rows",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    F.totalprice AS ending_price
                ALL ROWS PER MATCH WITH UNMATCHED ROWS
                PATTERN (A (B C | D E)+ F)
                DEFINE
                    B AS totalprice < PREV(totalprice),
                    C AS totalprice > PREV(totalprice),
                    D AS totalprice = PREV(totalprice),
                    E AS totalprice >= PREV(totalprice)
            );
        """
    },
    # 6. Using Classifier() and MATCH_NUMBER() functions
    {
        "name": "Classifier and Match Number Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    CLASSIFIER() AS pattern_type,
                    MATCH_NUMBER() AS match_id,
                    A.totalprice AS starting_price,
                    LAST(B.totalprice) AS bottom_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 7. Running vs Final Semantics in MEASURES
    {
        "name": "Running vs Final Semantics in Measures",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    RUNNING LAST(A.totalprice) AS running_last_price,
                    FINAL LAST(A.totalprice) AS final_last_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 8. Aggregates with Running vs Final semantics (avg and count)
    {
        "name": "Running Avg vs Final Count Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    RUNNING avg(A.totalprice) AS running_avg,
                    FINAL count(A.*) AS final_count
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 9. Empty Match: pattern that produces an empty match for every row
    {
        "name": "Empty Match and Unmatched Rows",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    A.totalprice AS starting_price,
                    count(A.*) AS match_count
                ONE ROW PER MATCH
                PATTERN (())
            );
        """
    },
    # 10. Physical Navigation Functions: PREV() and NEXT()
    {
        "name": "Physical Navigation Functions",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    PREV(A.totalprice, 2) AS prev_price,
                    NEXT(A.totalprice, 1) AS next_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    },
    # 11. Nested Navigation Function
    {
        "name": "Nested Navigation Function",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES
                    PREV(FIRST(A.totalprice, 3), 2) AS nested_nav_price
                ONE ROW PER MATCH
                PATTERN (A B+ C)
                DEFINE
                    B AS totalprice < PREV(totalprice)
            );
        """
    }
]

# Run all test cases and print the resulting AST
for test in test_queries:
    print(f"🔹 Running Test: {test['name']}")
    try:
        query_ast = parse_full_query(test["query"])
        print(query_ast)
    except Exception as e:
        print(f"Error: {e}")
    print("=" * 80)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY custkey ORDER BY orderdate MEASURES A.totalprice AS starting_price, LAST(B.totalprice) AS bottom_price ONE ROW PER MATCH PATTERN (A B+ C) DEFINE B AS totalprice < PREV(totalprice) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_

🔹 Running Test: Basic One Row Per Match
FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False), Measure(expression='LAST(B.totalprice)', alias='bottom_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=ONE ROW PER MATCH,
  after_match_skip=None,
  pattern=PatternClause(pattern='A B+ C', metadata={'variables': ['A', 'B+', 'C'], 'base_variables': ['A', 'B', 'C']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='B', condition='totalprice<PREV(totalprice)')])
),
  metadata

In [1]:
import logging

# Import your SQL parser function here; adjust the import as needed.
from src.parser.match_recognize_extractor import parse_full_query

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# A comprehensive list of test queries covering various SQL components after SELECT.
test_queries = [
    {
        "description": "Wildcard Column Selection",
        "query": "SELECT * FROM orders;"
    },
    {
        "description": "Specific Column Selection",
        "query": "SELECT orderid, custkey, orderdate FROM orders;"
    },
    {
        "description": "Expressions & Aliases",
        "query": "SELECT orderid, totalprice * 0.9 AS discounted_price FROM orders;"
    },
    {
        "description": "Aggregate Functions",
        "query": "SELECT COUNT(*) AS total_orders, SUM(totalprice) AS total_sales FROM orders;"
    },
    {
        "description": "WHERE Clause",
        "query": "SELECT orderid, totalprice FROM orders WHERE totalprice > 100;"
    },
    {
        "description": "ORDER BY Clause",
        "query": "SELECT orderid, orderdate FROM orders ORDER BY orderdate DESC;"
    },
    {
        "description": "GROUP BY & HAVING Clauses",
        "query": "SELECT custkey, COUNT(*) AS order_count FROM orders GROUP BY custkey HAVING COUNT(*) > 5;"
    },
    {
        "description": "LIMIT & OFFSET",
        "query": "SELECT * FROM orders LIMIT 10 OFFSET 20;"
    },
    {
        "description": "DISTINCT",
        "query": "SELECT DISTINCT custkey FROM orders;"
    },
    {
        "description": "JOIN",
        "query": "SELECT o.orderid, c.custname FROM orders o JOIN customers c ON o.custkey = c.custkey;"
    },
    {
        "description": "Subquery",
        "query": "SELECT orderid FROM orders WHERE totalprice > (SELECT AVG(totalprice) FROM orders);"
    },
    {
        "description": "UNION / UNION ALL",
        "query": "SELECT orderid FROM orders_2024 UNION ALL SELECT orderid FROM orders_2025;"
    },
    {
        "description": "CASE Expression",
        "query": "SELECT orderid, CASE WHEN totalprice > 1000 THEN 'High' ELSE 'Low' END AS price_category FROM orders;"
    }
]

def run_tests():
    for test in test_queries:
        print("\n--- Running Test: {} ---".format(test["description"]))
        try:
            ast = parse_full_query(test["query"])
            print("Test Passed: Query parsed successfully.")
            print("AST:", ast)
        except Exception as e:
            print("Test Failed with error:")
            print(e)
            print("Query:")
            print(test["query"])

if __name__ == "__main__":
    run_tests()


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders;
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted MATCH_RECOGNIZE clause via recursive search.
DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT orderid, custkey, orderdate FROM orders;
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=orderid, metadata={}), SelectItem(expression=custkey, metadata={}), SelectItem(expression=orderdate, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:


--- Running Test: Wildcard Column Selection ---
Test Passed: Query parsed successfully.
AST: FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=*, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=None,
  order_by=None,
  measures=None,
  rows_per_match=None,
  after_match_skip=None,
  pattern=None,
  subset=[],
  define=None
),
  metadata={}
)

--- Running Test: Specific Column Selection ---
Test Passed: Query parsed successfully.
AST: FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=orderid, metadata={}), SelectItem(expression=custkey, metadata={}), SelectItem(expression=orderdate, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=None,
  order_by=None,
  measures=None,
  rows_per_match=None,
  after_match_skip=None,
  pattern=None,
  subset=[],
  define=None
),
  metadata={}
)

--- Running Test: Expressions & Alias

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT COUNT(*) AS total_orders, SUM(totalprice) AS total_sales FROM orders;
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=COUNT(*), alias=total_orders, metadata={}), SelectItem(expression=SUM(totalprice), alias=total_sales, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted MATCH_RECOGNIZE clause via recursive search.
DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT orderid, totalprice FROM orders WHERE totalprice > 100;
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=orderid, metadata={}), SelectItem(expression=totalprice, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted 

Test Passed: Query parsed successfully.
AST: FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=COUNT(*), alias=total_orders, metadata={}), SelectItem(expression=SUM(totalprice), alias=total_sales, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=None,
  order_by=None,
  measures=None,
  rows_per_match=None,
  after_match_skip=None,
  pattern=None,
  subset=[],
  define=None
),
  metadata={}
)

--- Running Test: WHERE Clause ---
Test Passed: Query parsed successfully.
AST: FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=orderid, metadata={}), SelectItem(expression=totalprice, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=None,
  order_by=None,
  measures=None,
  rows_per_match=None,
  after_match_skip=None,
  pattern=None,
  subset=[],
  define=None
),
  metadata={}
)

--- Running Test: ORDER BY Clause ---
Test P

DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted MATCH_RECOGNIZE clause via recursive search.
DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT o.orderid, c.custname FROM orders o JOIN customers c ON o.custkey = c.custkey;
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=o.orderid, metadata={}), SelectItem(expression=c.custname, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted MATCH_RECOGNIZE clause via recursive search.
DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT orderid FROM orders WHERE totalprice > (SELECT AVG(totalprice) FROM orders);
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: Sel

Test Passed: Query parsed successfully.
AST: FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=DISTINCT custkey, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=None,
  order_by=None,
  measures=None,
  rows_per_match=None,
  after_match_skip=None,
  pattern=None,
  subset=[],
  define=None
),
  metadata={}
)

--- Running Test: JOIN ---
Test Passed: Query parsed successfully.
AST: FullQueryAST(
  select_clause=SelectClause(items=[SelectItem(expression=o.orderid, metadata={}), SelectItem(expression=c.custname, metadata={})]),
  from_clause=FromClause(table='orders'),
  match_recognize=MatchRecognizeClause(
  partition_by=None,
  order_by=None,
  measures=None,
  rows_per_match=None,
  after_match_skip=None,
  pattern=None,
  subset=[],
  define=None
),
  metadata={}
)

--- Running Test: Subquery ---
Test Passed: Query parsed successfully.
AST: FullQueryAST(
  select_clause=SelectClause(items=[SelectIt

In [2]:
from src.parser.match_recognize_extractor import parse_match_recognize_query
import logging

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

test_queries = [
    {
        "description": "Simple pattern with whitespace",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price
                PATTERN (A b+ c*)
                DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
            );
        """
    },
    {
        "description": "Pattern without whitespace",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price
                PATTERN (AB+C)
                DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
            );
        """
    },
    {
        "description": "Pattern with PERMUTE, grouping, and quantifiers",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES LAST(U.totalprice) AS top_price
                PATTERN (PERMUTE(A,B,C))
                SUBSET U = (A,B,C)
                DEFINE A AS totalprice > 0, B AS totalprice < PREV(totalprice), C AS totalprice < 100
            );
        """
    },
    {
        "description": "Mixed pattern: whitespace vs. no whitespace",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price
                PATTERN (A b+ c*)
                DEFINE A AS totalprice > 0, b AS totalprice < PREV(totalprice), c AS totalprice < 100
            );
        """
    },
    {
        "description": "Pattern with quantifiers (greedy and reluctant)",
        "query": """
            SELECT * FROM orders MATCH_RECOGNIZE(
                PARTITION BY custkey
                ORDER BY orderdate
                MEASURES A.totalprice AS starting_price, LAST(A.totalprice) AS ending_price
                PATTERN (A{2,4}? )
                DEFINE A AS totalprice > 0
            );
        """
    },
]

def run_tests():
    for test in test_queries:
        print("\n--- Running Test: {} ---".format(test["description"]))
        try:
            ast = parse_match_recognize_query(test["query"])
            print("Test Passed: Query parsed successfully.")
            print(ast)
        except Exception as e:
            print("Test Failed with error:")
            print(e)
            print("Query:")
            print(test["query"])

if __name__ == "__main__":
    run_tests()


DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['custkey'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)])
DEBUG:src.parser.match_recognize_extractor:Updated Pattern tokens: {'variables': ['A', 'b+', 'c*'], 'base_variables': ['A', 'b', 'c']}
DEBUG:src.parser.match_recognize_extractor:PATTERN clause validated successfully: A b+ c*
DEBUG:src.parser.match_recognize_extractor:Extracted Pattern: PatternClause(pattern='A b+ c*', metadata={'variables': ['A', 'b+', 'c*'], 'base_variables': ['A', 'b', 'c']})
DEBUG:src.parser.match_recognize


--- Running Test: Simple pattern with whitespace ---
Test Passed: Query parsed successfully.
MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']),
  order_by=OrderByClause(sort_items=[SortItem(column='orderdate', ordering='ASC', nulls_ordering=None)]),
  measures=MeasuresClause(measures=[Measure(expression='A.totalprice', alias='starting_price', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=False)]),
  rows_per_match=None,
  after_match_skip=None,
  pattern=PatternClause(pattern='A b+ c*', metadata={'variables': ['A', 'b+', 'c*'], 'base_variables': ['A', 'b', 'c']}),
  subset=[],
  define=DefineClause(definitions=[Define(variable='A', condition='totalprice>0'), Define(variable='b', condition='totalprice<PREV(totalprice)'), Define(variable='c', condition='totalprice<100')])
)

--- Running Test: Pattern without whitespace ---
Test Passed: Query parsed successfully.
MatchRecognizeClause(
  partition_by=PartitionByClause(columns=['custkey']